In [7]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201 
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, log_loss, jaccard_score
import numpy as np
import os
from PIL import Image
from shutil import copyfile  # Import the copyfile function
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

2024-04-29 17:04:43.207852: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# Define the paths to our train, validation, and test datasets
train_data_dir =  '/tf/Datasets/BoneFractureDataset/training'
test_data_dir = '/tf/Datasets/BoneFractureDataset/testing'
validation_data_dir = '/tf/Datasets/BoneFractureDataset/testing'

# Image dimensions
IMG_WIDTH, IMG_HEIGHT = 299, 299
input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)  # RGB images

## Data generators

In [33]:
BATCH_SIZE = 10

In [43]:
# Define data generators for RGB images with augmentation
datagen_augmented = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  # Rotation angle range
    width_shift_range=0.2,  # Horizontal shift range
    height_shift_range=0.2,  # Vertical shift range
    shear_range=0.2,  # Shear intensity
    zoom_range=0.2,  # Random zoom range
    horizontal_flip=True,  # Randomly flip images horizontally
    vertical_flip=False,  # Do not flip images vertically
    fill_mode='nearest'  # Fill mode for newly created pixels
)
# Generate augmented data for training
train_generator = datagen_augmented.flow_from_directory(
    train_data_dir,
    batch_size=BATCH_SIZE,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    class_mode='categorical',  
)

Found 8863 images belonging to 2 classes.


In [44]:
images, labels = next(train_generator)
print(images.dtype, images.shape)
print(labels.dtype, labels.shape)

float32 (10, 299, 299, 3)
float32 (10, 2)


In [45]:
# Create a train dataset
train_ds = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=(images.shape, labels.shape)
)
train_ds.element_spec

(TensorSpec(shape=(10, 299, 299, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(10, 2), dtype=tf.float32, name=None))

In [47]:
datagen_noaugment = ImageDataGenerator(rescale=1./255)

test_generator = datagen_noaugment.flow_from_directory(
    test_data_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
)

# Create a test dataset
test_ds = tf.data.Dataset.from_generator(
    lambda: test_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=(images.shape, labels.shape)
)
test_ds.element_spec

Found 600 images belonging to 2 classes.


(TensorSpec(shape=(10, 299, 299, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(10, 2), dtype=tf.float32, name=None))

In [48]:
valid_generator = datagen_noaugment.flow_from_directory(
    validation_data_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
)

# Create a test dataset
valid_ds = tf.data.Dataset.from_generator(
    lambda: valid_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=(images.shape, labels.shape)
)
valid_ds.element_spec

Found 600 images belonging to 2 classes.


(TensorSpec(shape=(10, 299, 299, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(10, 2), dtype=tf.float32, name=None))

In [10]:
train_generator.class_indices

{'fractured': 0, 'not_fractured': 1}

### Number of images for each class in the datasets

In [22]:
# Count the number of images for each class in the training dataset
for data_dir in [train_data_dir, test_data_dir, validation_data_dir]:
    classes = os.listdir(data_dir)
    for class_name in classes:
        class_path = os.path.join(data_dir, class_name)
        num_images = len(os.listdir(class_path))
        print(f"{data_dir}_Class: {class_name}, Number of images: {num_images}")

/tf/Datasets/BoneFractureDataset/training_Class: not_fractured, Number of images: 4383
/tf/Datasets/BoneFractureDataset/training_Class: fractured, Number of images: 4480
/tf/Datasets/BoneFractureDataset/testing_Class: not_fractured, Number of images: 240
/tf/Datasets/BoneFractureDataset/testing_Class: fractured, Number of images: 360
/tf/Datasets/BoneFractureDataset/testing_Class: not_fractured, Number of images: 240
/tf/Datasets/BoneFractureDataset/testing_Class: fractured, Number of images: 360


### Check the shape of the images in Train Generator

In [49]:
for images, labels in train_ds.take(1):
    print(f'{images.shape}, {labels.shape}')
    break

(10, 299, 299, 3), (10, 2)


### Check GPU

In [23]:
# Check for GPU availability
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT available")

# Set TensorFlow to use the GPU device
if tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
    print("GPU device configured")
else:
    print("No GPU device found")

GPU is available
GPU device configured


2024-04-29 17:33:57.731610: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-29 17:33:57.737418: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-29 17:33:57.737537: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

## Model Checkpoint

In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint
model_dir = './working/Checkpoints_densenet201'

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

checkpoint_path = model_dir + '/densenet201.weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_best_only=True,  # Save only the best model
    monitor="val_accuracy",   # Monitor validation loss
    mode="max",           # Save the model when validation loss is minimized
    verbose=1
)

In [27]:
checkpoint_path

'./working/Checkpoints_densenet201/densenet201.weights.h5'

## DenseNet for Feature Extractor

In [28]:
from tensorflow.keras import models, layers, optimizers

In [29]:
def create_model(summary=True):
    # apply transfer learning
    new_input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    base_model = DenseNet201(weights='imagenet', include_top=False, input_tensor=new_input) ##MobileNetV3Small(weights='imagenet', include_top=False, input_tensor=new_input)
    # add new classifier layers
    flat1 = Flatten()(base_model.layers[-1].output)
    output = Dense(2, activation='softmax')(flat1)  
    # define new model
    model = Model(inputs=base_model.inputs, outputs=output)
    # Modify loss function to 'weighted_binary_crossentropy'
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    if summary:
        print(model.summary())
    return model

### Model summary

In [30]:
model = create_model()

2024-04-29 17:41:15.823028: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-29 17:41:15.823138: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-29 17:41:15.823252: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 305, 305,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 150, 150,  │      9,408 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 150, 150,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 150, 150,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 152, 152,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 75, 75,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 75, 75,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 75, 75,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 75, 75,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 75, 75,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 75, 75,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 75, 75,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 75, 75,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 75, 75,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 75, 75,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 75, 75,    │     12,288 │ conv2_block2_0_r

 Total params: 18,633,026 (71.08 MB)

 Trainable params: 18,403,970 (70.21 MB)

 Non-trainable params: 229,056 (894.75 KB)

None


### Training

In [53]:
# Train the model
history = model.fit(
    train_ds.repeat(),
    steps_per_epoch=100,
    epochs=100,
    validation_data=valid_ds.repeat(),
    validation_steps=25,
    callbacks=[cp_callback]
    
)

Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.6658 - loss: 0.6077
Epoch 1: val_accuracy did not improve from 0.77500
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 89ms/step - accuracy: 0.6659 - loss: 0.6076 - val_accuracy: 0.5640 - val_loss: 0.7725
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.6961 - loss: 0.5787
Epoch 2: val_accuracy did not improve from 0.77500
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 87ms/step - accuracy: 0.6961 - loss: 0.5787 - val_accuracy: 0.5440 - val_loss: 0.7491
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.7035 - loss: 0.5558
Epoch 3: val_accuracy did not improve from 0.77500
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.7035 - loss: 0.5558 - val_accuracy: 0.5880 - val_loss: 0.9199
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.6819 - loss: 0.5804
Epoch 4: val_accuracy improved from 0.77500 to 0.80000, saving model to ./working/Checkpoints_densenet201/densenet201.weights.h5
100/100 ━━━━

2024-04-29 18:29:22.198356: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (3, 299, 299, 3) where an element of shape (10, 299, 299, 3) was expected.
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (3, 299, 299, 3) where an element of shape (10, 299, 299, 3) was expected.




 82/100 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.6847 - loss: 0.5732

2024-04-29 18:29:22.408810: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (3, 299, 299, 3) where an element of shape (10, 299, 299, 3) was expected.
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (3, 299, 299, 3) where an element of shape (10, 299, 299, 3) was expected.


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]]
2024-04-29 18:29:22.409536: W tensorflow/co

InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  TypeError: `generator` yielded an element of shape (3, 299, 299, 3) where an element of shape (10, 299, 299, 3) was expected.
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (3, 299, 299, 3) where an element of 